# Analysis of supervised models

In [ ]:
import os

import numpy as np
import pandas as pd

figure_dir = os.path.join(os.getcwd(), '..', '..', 'figures', 'supervised')
if not os.path.exists(figure_dir):
    os.makedirs(figure_dir)

evaluation_dir = os.path.join(os.getcwd(), '..', '..', 'evaluation')
predictions_file = os.path.join(evaluation_dir, 'pretrained', 'predictions.csv')

predictions = pd.read_csv(predictions_file)
predictions['signal'] = predictions['signal'].apply(eval).apply(np.array)
predictions['duration'] = pd.to_timedelta(predictions['duration'])

ground_truth_file = os.path.join(evaluation_dir, 'ground_truth.csv')
ground_truth = pd.read_csv(ground_truth_file)
ground_truth['signal'] = ground_truth['signal'].apply(eval).apply(np.array)

predictions.head()

## Compare the predictions with the ground truth

In [ ]:
import respiration.analysis as analysis
import tqdm.auto as tqdm

analysis_file = os.path.join(evaluation_dir, 'pretrained', 'analysis.csv')

if os.path.exists(analysis_file):
    analysis_results = pd.read_csv(analysis_file)
else:
    analysis_results = []

    for index, row in tqdm.tqdm(predictions.iterrows(), total=len(predictions)):
        subject, setting = row['subject'], row['setting']

        # Get the ground truth signal and sampling rate
        gt_signal, gt_sampling_rate = ground_truth[
            (ground_truth['subject'] == subject) &
            (ground_truth['setting'] == setting)
            ][['signal', 'sample_rate']].values[0]

        prediction, sampling_rate = row['signal'], row['sample_rate']

        compare = analysis.SignalCompare(
            prediction,
            sampling_rate,
            gt_signal,
            gt_sampling_rate,
        )

        for metric, result in compare.distances().items():
            analysis_results.append({
                'subject': subject,
                'setting': setting,
                'model': row['model'],
                'metric': metric,
                'result': result,
            })

    analysis_results = pd.DataFrame(analysis_results)
    analysis_results.to_csv(analysis_file, index=False)

In [ ]:
analysis_results

## Analyze the results

In [ ]:
method_results = []

metrics = analysis_results['metric'].unique()
models = analysis_results['model'].unique()

for model in models:
    for metric in metrics:
        data = analysis_results[
            (analysis_results['model'] == model) &
            (analysis_results['metric'] == metric)
            ]

        method_results.append({
            'model': model,
            'metric': metric,
            'mean': data['result'].mean(),
            'std': data['result'].std(),
        })

method_results = pd.DataFrame(method_results)

In [ ]:
method_results

In [ ]:
# Plot for each metric how well the methods performs
import matplotlib.pyplot as plt

metrics = [
    'pk_error',
    'cp_error',
    'nfcp_error',
    'fft_error',
    'distance_mse',
    'distance_pearson',
    'distance_dtw',
]
metrics.sort()

fig, axes = plt.subplots(3, 3, figsize=(20, 7))

# Add some space between the plots
fig.tight_layout(pad=5.0)

fig.suptitle('Comparison of supervised methods')

for idx, metric in enumerate(metrics):
    ax = axes[idx // 3, idx % 3]
    data = method_results[method_results['metric'] == metric]

    # Plot the mean and standard deviation
    ax.bar(data['model'], data['mean'], yerr=data['std'], capsize=5)

    ax.set_title(metric)
    ax.set_ylabel('Mean distance')
    ax.set_xlabel('Model')

    # Rotate the x-axis labels
    for tick in ax.get_xticklabels():
        tick.set_rotation(0)

# Store the plot as svg
plot_file = os.path.join(figure_dir, 'error_method_comparison.svg')
plt.savefig(plot_file, format='svg')